In [1]:
from GensimResearchCorpus import ResearchCorpus, GensimResearchCorpus
import gensim
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [15]:
es = Elasticsearch([{
    'host':'localhost', 
    'http_auth':('elastic', 'Today123'),
    'port': 9200}])
model = gensim.models.doc2vec.Doc2Vec.load('doc2vec.model_100_5_cs')

In [3]:
totalCount=0
for researchItem in ResearchCorpus(filterCategory='cs'):
    totalCount+=1
print(totalCount)
print(researchItem)

535636
{'id': 'solv-int/9906012', 'submitter': 'S. Yu. Sakovich', 'authors': 'S. Yu. Sakovich', 'title': 'Integrability of the higher-order nonlinear Schroedinger equation   revisited', 'comments': '6 pages, LaTeX', 'journal-ref': None, 'doi': None, 'report-no': None, 'categories': 'solv-int math-ph math.AP math.MP nlin.SI physics.optics', 'license': None, 'abstract': 'Only the known integrable cases of the Kodama-Hasegawa higher-order nonlinear Schroedinger equation pass the Painleve test. Recent results of Ghosh and Nandy add no new integrable cases of this equation.', 'versions': [{'version': 'v1', 'created': 'Wed, 23 Jun 1999 05:51:22 GMT'}], 'update_date': '2007-05-23', 'authors_parsed': [['Sakovich', 'S. Yu.', '']], 'abstract_cleaned': 'only the known integrable cases of the kodama-hasegawa higher-order nonlinear schroedinger equation pass the painleve test. recent results of ghosh and nandy add no new integrable cases of this equation.', 'tokens': ['only', 'the', 'known', 'integ

In [6]:
data=[]
for researchItem in ResearchCorpus(filterCategory='cs'):
    doc2vec_vector=model.infer_vector(researchItem['tokens'])
    mydict={
        'id': researchItem['id'],
        'submitter': researchItem['submitter'],
        'update_date': researchItem['update_date'],
        'authors': researchItem['authors'],
        'title': researchItem['title'],
        'journal-ref': researchItem['journal-ref'],
        'doi': researchItem['doi'],
        'categories': researchItem['categories'],
        'abstract': researchItem['abstract'],
        'abstract_vector': doc2vec_vector,
    }
    data.append(mydict)
    if len(data)>=100:
        bulk(es, data, index="arxiv")
        data=[]
bulk(es, data, index="arxiv")

(36, [])

In [ ]:
username = 'elastic'
password = 'Today123'

In [31]:
import requests
import json

url = "http://localhost:9200/arxiv/_search"

payload = json.dumps({
  "query": {
    "multi_match": {
      "query": "0704.0002",
        "fields": ["id"]
    }
  }
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, auth = HTTPBasicAuth(username, password))

result=json.loads(response.text)
result["hits"]["hits"][0]["_source"]["abstract"]

'We describe a new algorithm, the $(k,\\ell)$-pebble game with colors, and use it obtain a characterization of the family of $(k,\\ell)$-sparse graphs and algorithmic solutions to a family of problems concerning tree decompositions of graphs. Special instances of sparse graphs appear in rigidity theory and have received increased attention in recent years. In particular, our colored pebbles generalize and strengthen the previous results of Lee and Streinu and give a new proof of the Tutte-Nash-Williams characterization of arboricity. We also present a new decomposition that certifies sparsity based on the $(k,\\ell)$-pebble game with colors. Our work also exposes connections between pebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and Westermann and Hendrickson.'

In [30]:
payload = json.dumps({
  "query": {
      "script_score": {
        "query": {"match_all": {}},
        "script": {
          "source": "cosineSimilarity(params.query_vector, 'abstract_vector') + 1.0",
          "params": {"query_vector": result["hits"]["hits"][0]["_source"]["abstract_vector"]}
        }
      }
    }
})



headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload, auth = HTTPBasicAuth(username, password))

results=json.loads(response.text)
for item in results['hits']['hits']:
    print(item['_source']['abstract'])

We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use it obtain a characterization of the family of $(k,\ell)$-sparse graphs and algorithmic solutions to a family of problems concerning tree decompositions of graphs. Special instances of sparse graphs appear in rigidity theory and have received increased attention in recent years. In particular, our colored pebbles generalize and strengthen the previous results of Lee and Streinu and give a new proof of the Tutte-Nash-Williams characterization of arboricity. We also present a new decomposition that certifies sparsity based on the $(k,\ell)$-pebble game with colors. Our work also exposes connections between pebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and Westermann and Hendrickson.
Starting with a graph, two players take turns in either deleting an edge or deleting a vertex and all incident edges. The player removing the last vertex wins. We review the known results for this game an

In [25]:
zz=json.loads(response.text)
len(zz['hits']['hits'])

10

In [20]:
import requests
from requests.auth import HTTPBasicAuth 
from pprint import pprint 



response = requests.get('http://localhost:9200/arxiv/_search', auth = HTTPBasicAuth(username, password))

pprint(response.content)


(b'{"took":9,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,'
 b'"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":1.0'
 b',"hits":[{"_index":"arxiv","_type":"_doc","_id":"GP5DWnoBVdEtFzgbwZRw","_sco'
 b're":1.0,"_source":{"id":"0705.1671","submitter":"Valerio Biancalana","update'
 b'_date":"2014-11-13","authors":"J.Belfi, G.Bevilacqua, V.Biancalana, Y.Danche'
 b'va, L.Moi","title":"All optical sensor for automated magnetometry based on C'
 b'oherent   Population Trapping","journal-ref":"JOSA B 24 (7) (2007), pages 14'
 b'82-1489","doi":"10.1364/JOSAB.24.001482","categories":"physics.atom-ph physi'
 b'cs.ins-det","abstract":"An automated magnetometer suitable for long lasting '
 b'measurement under stable and controllable experimental conditions has been i'
 b'mplemented. The device is based on Coherent Population Trapping (CPT) produc'
 b'ed by a multi-frequency excitation. CPT resonance is observed when a frequen'
 b'cy comb, generated by dio